In [80]:
import datetime as dt
import sys

import numpy as np
import pandas as pd
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from sklearn.preprocessing import MinMaxScaler
from scipy import signal

init_notebook_mode(connected=True)

sys.path.insert(0, '../')
import gps

df = pd.read_csv('demos/some_data.csv', parse_dates=['ts'])
sclr = MinMaxScaler((0, 2*np.pi))

df = gps.process_velocities(df)

In [88]:
def seconds_into_day(t):
    d = dt.datetime(year=t.year, month=t.month, day=t.day, hour=0, minute=0, second=0)
    return (t-d).total_seconds()

df['day'] = df.ts.apply(lambda x: x.date())
df['second_of_day'] = df.ts.apply(seconds_into_day)
df['hod'] = df.ts.apply(lambda x: x.hour)
df['freq'] = np.sin(df.second_of_day)

df['lat_power'] = signal.lombscargle(df.ts, df.velocity, df.freq)
# df['lon_power'] = signal.lombscargle(df.ts, df.lon_freq, df.second_of_day)
# df['power'] = [np.log(t.lat_power + t.lon_power) for t in df.itertuples()]
df['i'] = [(t.day - np.min(df.day)).days for t in df.itertuples()]

df.head()

,lat,lon,ts,binning,displacement,time_delta,velocity,day,second_of_day,hod,i,freq,lat_power
0,47.66133,-122.29343,2018-06-07 00:00:08.668,stationary,NaN,NaN,NaN,2018-06-07,8.668,0,0,0.686582,NaN
1,47.66123,-122.29655,2018-06-07 00:00:24.000,powered_vehicle,233.9,15.0,15.595,2018-06-07,24.000,0,0,-0.905578,NaN
2,47.66125,-122.29673,2018-06-07 00:00:25.000,powered_vehicle,13.7,1.0,13.663,2018-06-07,25.000,0,0,-0.132352,NaN
3,47.66087,-122.29904,2018-06-07 00:00:40.000,powered_vehicle,178.1,15.0,11.872,2018-06-07,40.000,0,0,0.745113,NaN
4,47.66067,-122.29951,2018-06-07 00:00:56.000,active,41.6,16.0,2.602,2018-06-07,56.000,0,0,-0.521551,NaN


In [76]:
iplot(go.Figure([
    go.Scatter(
        x=df.ts, 
        y=df.lat,
#        marker=dict(
#            color=df.lat_power, 
#            colorscale='Viridis',  
#            colorbar=dict(title='Power'),
#            opacity=MinMaxScaler((.5,1)).fit_transform(np.array(df.lat_power).reshape((-1, 1))),
#        )
      ),
    go.Scatter(
        x=df.ts,
        y=df.lon,
        yaxis='y2',
    )
],
    go.Layout(
        yaxis2=dict(overlaying='y', side='right'),
        legend=dict(orientation='h')
    )
))

In [86]:
iplot(go.Figure([
    go.Scatter(
        x=df.ts,
        y=df.displacement,
        mode='markers'
    ),
    go.Scatter(
        x=df.ts
    )
]))

In [ ]:
t = df[['day', 'hod', 'ts', 'power']].groupby(['day', 'hod']).mean().reset_index()
t.rolling

In [ ]:
daily.head()
iplot(go.Figure([
    go.Scatter(
        x=df.ts,
        y=df.power
    )
]))